In [51]:
#all imports

import pandas as pd
import seaborn as sb
import math
import numpy as np
import os
import re
import jenkspy
from ast import literal_eval
from glob import glob
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, _tree


import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder

from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score, ShuffleSplit
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn import preprocessing

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.metrics import classification_report


from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [22]:
df1 = pd.read_csv("df_mfeatures.csv")
df2 = pd.read_csv("df_ensemble_allfeatures.csv")

binDF = pd.read_csv("BinnedMetaFeatures.csv")

In [23]:
df_mfeatures = df1[['File', 'Completeness', 'Conciseness', 'cor.mean', 'cov.mean',
       'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 'kurtosis.mean', 
       'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr',
       'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean',
       'sparsity.mean', 't_mean.mean',   'var.mean', 'ClassImbRatio',
       'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr',
        'nr_bin',  'nr_inst', 'nr_num', 'attr_conc.mean', 'attr_ent.mean', 'ena', 'nUnique' ,'snr.mean', 'cEntropy','LabelIssuesPerc']] #SyntaxAccuracy removed

# Majority voting

In [27]:
#Needs to be run only one - writes majority voted labels into a file
path = os.getcwd()+'/optimalalphabeta/dearr_cont/'  
csv_files = glob(os.path.join(path, 'dearr*.csv'))

labels_dict = {}

for file in csv_files:
    df = pd.read_csv(file)
    
    for idx, row in df.iterrows():
        dataset_id = row['File']  
        best_fs = row['FeatureAlgo']     
        
        if dataset_id not in labels_dict:
            labels_dict[dataset_id] = []
        labels_dict[dataset_id].append(best_fs)

majority_labels = {
    dataset: Counter(fs_list).most_common(1)[0][0]
    for dataset, fs_list in labels_dict.items()
}

majority_df = pd.DataFrame(list(majority_labels.items()), columns=['File', 'FeatureAlgo'])
majority_df.to_csv('majority_voted_labels.csv', index=False)

# Majority voting based on a cluster

In [31]:
path = os.getcwd()+'/optimalalphabeta/dearr_cont/'  
label_column = 'FeatureAlgo'        
output_dim = 2                   
n_clusters = 4                   

csv_files = sorted(glob(os.path.join(path, 'dearr*.csv')))

all_fs_algos = set()
fs_distributions = []
ab_values = []

for file in csv_files:
    df = pd.read_csv(file)
    fs_counts = Counter(df[label_column])
    all_fs_algos.update(fs_counts.keys())
    fs_distributions.append(fs_counts)
    
    # Extract alpha, beta from filename like 'dearr0.02_0.04.csv'
    ab = file.split('/')[-1].replace('dearr', '').replace('.csv', '')
    alpha, beta = map(float, ab.split('_'))
    ab_values.append((alpha, beta))

fs_algos = sorted(list(all_fs_algos))

matrix = []
for dist in fs_distributions:
    vector = [dist.get(fs, 0) for fs in fs_algos]
    matrix.append(vector)

df_matrix = pd.DataFrame(matrix, columns=fs_algos)
df_matrix.index = [f'{a}_{b}' for a, b in ab_values]

scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(df_matrix)

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(scaled_matrix)

cluster_df = pd.DataFrame({
    'File': df_matrix.index,  # e.g., "0.02_0.04"
    'Cluster': labels
})

cluster_counts = cluster_df['Cluster'].value_counts().sort_index()

print("Number of alpha-beta files in each cluster:")
print(cluster_counts)

print("\nDetailed files per cluster:")
for cluster_id, group in cluster_df.groupby('Cluster'):
    print(f"\nCluster {cluster_id} ({len(group)} files):")
    print(group['File'].tolist())



Number of alpha-beta files in each cluster:
0     5
1    12
2    18
3     1
Name: Cluster, dtype: int64

Detailed files per cluster:

Cluster 0 (5 files):
['0.0_0.02', '0.0_0.04', '0.0_0.06', '0.0_0.08', '0.0_0.1']

Cluster 1 (12 files):
['0.02_0.0', '0.02_0.02', '0.02_0.04', '0.02_0.06', '0.02_0.08', '0.02_0.1', '0.04_0.0', '0.04_0.02', '0.04_0.04', '0.04_0.06', '0.04_0.08', '0.04_0.1']

Cluster 2 (18 files):
['0.06_0.0', '0.06_0.02', '0.06_0.04', '0.06_0.06', '0.06_0.08', '0.06_0.1', '0.08_0.0', '0.08_0.02', '0.08_0.04', '0.08_0.06', '0.08_0.08', '0.08_0.1', '0.1_0.0', '0.1_0.02', '0.1_0.04', '0.1_0.06', '0.1_0.08', '0.1_0.1']

Cluster 3 (1 files):
['0.0_0.0']


# Generate majority vote based on cluster

The files are copied into ./optimalalphabeta/clusteredearr/ directory

In [32]:

path = os.getcwd()+'/optimalalphabeta/clusteredearr/'  # replace with the path to your 36 CSVs
csv_files = glob(os.path.join(path, 'dearr*.csv'))

labels_dict = {}

for file in csv_files:
    df = pd.read_csv(file)
    
    for idx, row in df.iterrows():
        dataset_id = row['File']  # replace 'Dataset' with your actual column name
        best_fs = row['FeatureAlgo']     # replace 'Best_FS' with your actual column name
        
        if dataset_id not in labels_dict:
            labels_dict[dataset_id] = []
        labels_dict[dataset_id].append(best_fs)

majority_labels = {
    dataset: Counter(fs_list).most_common(1)[0][0]
    for dataset, fs_list in labels_dict.items()
}

majority_df = pd.DataFrame(list(majority_labels.items()), columns=['File', 'FeatureAlgo'])
majority_df.to_csv('majority_voted_labels_clusteredearr.csv', index=False)

Apply the following

In [101]:
def maptype(df):
    fs_mapping = {
    'MI': ('Uni', 'Information'),
    'GR': ('Uni', 'Information'),
    'fcbf': ('Multi', 'Information'),
    'chisquare': ('Uni', 'Dependency'),
    'mrmr': ('Multi', 'Dependency'),
    'cfs': ('Multi', 'Dependency'),
    'relief': ('Uni', 'Distance'),
    'relieff': ('Uni', 'Distance'),
    'multisurf': ('Multi', 'Distance')
    }

    def get_setting(algo):
        return fs_mapping.get(algo, ('Unknown', 'Unknown'))  # default if not found

    df[['FS_Setting', 'FS_Type']] = df['FeatureAlgo'].apply(lambda x: pd.Series(get_setting(x)))
    return df

In [41]:
def fst1_correlation(df1, df2, threshold=0.9):

    def correlation_filter(df, threshold):
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
        return df.drop(columns=to_drop)
    
    file_col = df2['File']

    X_filtered = correlation_filter(df2.drop(columns=['File']), threshold)

    X_filtered['File'] = file_col

    X_filtered = pd.merge(X_filtered, df1 , on='File', how='inner')  # use 'left', 'right', or 'outer' as needed
    return X_filtered.columns

In [42]:
def fst2_rf(df1):
    le = LabelEncoder()
    X = df1.drop(['File', 'FeatureAlgo','FS_Setting','FS_Type'], axis=1)
    X = X.apply(le.fit_transform)
    y= df1['FeatureAlgo']
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X, y)

    selector = SelectFromModel(clf, threshold='mean', prefit=True)
    X_selected = selector.transform(X)

    selected_features = X.columns[selector.get_support()]
    return selected_features.tolist()

In [45]:
df1 = pd.read_csv('majority_voted_labels_clusteredearr.csv')
df2 = pd.read_csv('meta-features-cont.csv')
fst1_features = fst1_correlation(df1, df2)


merged_df = pd.merge(df1, df2, on='File', how='inner')  # use 'left', 'right', or 'outer' as needed
merged_df.to_csv("majorityvoteclusteredearrdataset.csv",index=False)
merged_df = maptype(merged_df)
fst2_features = fst2_rf(merged_df)

common_features = list(set(fst1_features.tolist()).intersection(set(fst2_features)))


# Rule mining

In [48]:
def get_rules_with_support_confidence(tree_model, feature_names, X, y):
    tree_ = tree_model.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    rules = []

    def recurse(node, path, conds):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path = path.copy()
            left_path.append(f"({name} <= {threshold:.3f})")
            recurse(tree_.children_left[node], left_path, conds)

            right_path = path.copy()
            right_path.append(f"({name} > {threshold:.3f})")
            recurse(tree_.children_right[node], right_path, conds)
        else:
            path_str = " and ".join(path)
            support = tree_.n_node_samples[node] / tree_.n_node_samples[0]
            value = tree_.value[node][0]
            predicted_class = np.argmax(value)
            confidence = value[predicted_class] / np.sum(value)
            rules.append((path_str, predicted_class, support, confidence))

    recurse(0, [], rules)

    return pd.DataFrame(rules, columns=["Rule", "Predicted_Class", "Support", "Confidence"])

In [86]:
#With clustering
le = LabelEncoder()
X = merged_df[common_features]
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, train_size=0.8)

clf = DecisionTreeClassifier(max_depth=20, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting


y_type_pred = clf.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))

rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))


Feature_Algo Report (common features):
              precision    recall  f1-score   support

          GR       0.42      0.71      0.53         7
          MI       0.00      0.00      0.00         3
   chisquare       0.61      0.65      0.63        17
        fcbf       1.00      0.40      0.57         5
      relief       0.00      0.00      0.00         3

    accuracy                           0.51        35
   macro avg       0.41      0.35      0.35        35
weighted avg       0.52      0.51      0.49        35

                                                 Rule  Predicted_Class  \
0   (LabelIssuesPerc <= 27.500) and (cEntropy <= 5...                3   
1   (LabelIssuesPerc <= 27.500) and (cEntropy <= 5...                4   
29  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                4   
30  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                0   
31  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                4   
32  (LabelIssuesPerc > 27.50

In [87]:
rule_df.to_csv("rules.csv", index=False)

In [88]:
#With clustering
le = LabelEncoder()
X = merged_df[common_features]
X = X.apply(le.fit_transform)

y = merged_df['FS_Type'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, train_size=0.8)

clf = DecisionTreeClassifier(max_depth=20, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting


y_type_pred = clf.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))

rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))


Feature_Algo Report (common features):
              precision    recall  f1-score   support

  Dependency       0.72      0.76      0.74        17
    Distance       0.00      0.00      0.00         3
 Information       0.69      0.60      0.64        15

    accuracy                           0.63        35
   macro avg       0.47      0.45      0.46        35
weighted avg       0.65      0.63      0.64        35

                                                 Rule  Predicted_Class  \
0   (LabelIssuesPerc <= 27.500) and (cEntropy <= 5...                0   
29  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                1   
22  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                0   
23  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                2   
24  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                0   
25  (LabelIssuesPerc > 27.500) and (sparsity.mean ...                2   
26  (LabelIssuesPerc > 27.500) and (sparsity.mean ...         

In [89]:
rule_df.to_csv("rules_fstype.csv", index=False)